Profesores: Ing. Remigio Hurtado
Estudiantes: John Chiqui, Martin Bojorque
Materia: Aprendizaje de Maquina


# Practica 3 Redes Neuronales
## Descripción del problema y dataset
Este trabajo contiene datos para la estimación de los niveles de obesidad en personas de los países de México, Perú y Colombia, con edades entre 14 y 61 años y diversos hábitos alimentarios y condición física como lo menciona , los datos se recolectaron utilizando una plataforma web con una encuesta (ver Tabla 1) donde usuarios anónimos respondieron cada pregunta, luego se procesó la información obteniendo 17 atributos y 2111 registros Los atributos relacionados con los hábitos alimentarios son: Consumo frecuente de alimentos altos en calorías (FANG), Frecuencia de consumo de verduras (FCVC), Número de comidas principales (NCP), Consumo de alimentos entre comidas (CAEC), Consumo de agua diario (CH2O) y Consumo de alcohol (CALC). Los atributos relacionados con la condición física son: Monitoreo del consumo de calorías (SCC), Frecuencia de actividad física (PAP), Tiempo usando dispositivos tecnológicos (TUE), Transporte usado (MTRANS), otras variables obtenidas fueron: Sexo, Edad, Altura y Peso. Finalmente, se etiquetaron todos los datos y se creó la variable de clase NObesidad con los valores de Peso Insuficiente, Peso Normal, Sobrepeso Nivel I, Sobrepeso Nivel II, Obesidad Tipo I, Obesidad Tipo II y Obesidad Tipo III, con base en la Ecuación y información de la OMS y Normatividad mexicana.
## Preparación de datos (limpieza y preprocesamiento)

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.compose import make_column_transformer, ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
import keras
from keras.layers import Dense
from keras.models import Sequential
from keras.models import model_from_json
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
from keras.layers import Dropout
from tensorflow.python.keras.utils import data_utils

In [ ]:
url='https://raw.githubusercontent.com/gorbo96/RedNeuronal1/main/ObesityDataSet_raw_and_data_sinthetic.csv'
datos=pd.read_csv(url,sep=",")
datos

In [ ]:
categorico=['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','CALC','MTRANS']
numerico=['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']
auxiliar=['Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']
preprocesador = make_column_transformer(    
    (StandardScaler(),numerico),(OneHotEncoder(),categorico),('drop',['NObeyesdad'])
    )
DataSet=preprocesador.fit_transform(datos)
nombres = preprocesador.transformers_[1][1].get_feature_names(categorico)
auxiliar.extend(nombres)
DatasetPreprocesado = pd.DataFrame(data=DataSet,columns=auxiliar)
DatasetPreprocesado.head()

,Age,Height,Weight,FCVC,NCP,CH2O,FAF,TUE,Gender_Female,Gender_Male,family_history_with_overweight_no,family_history_with_overweight_yes,FAVC_no,FAVC_yes,CAEC_Always,CAEC_Frequently,CAEC_Sometimes,CAEC_no,SMOKE_no,SMOKE_yes,CALC_Always,CALC_Frequently,CALC_Sometimes,CALC_no,MTRANS_Automobile,MTRANS_Bike,MTRANS_Motorbike,MTRANS_Public_Transportation,MTRANS_Walking
0,-0.522124,-0.875589,-0.862558,-0.785019,0.404153,-0.013073,-1.188039,0.561997,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
1,-0.522124,-1.947599,-1.168077,1.088342,0.404153,1.618759,2.339750,-1.080625,1.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0
2,-0.206889,1.054029,-0.366090,-0.785019,0.404153,-0.013073,1.163820,0.561997,0.0,1.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0
3,0.423582,1.054029,0.015808,1.088342,0.404153,-0.013073,1.163820,-1.080625,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0
4,-0.364507,0.839627,0.122740,-0.785019,-2.167023,-0.013073,-1.188039,-1.080625,0.0,1.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


In [ ]:
eliminar=['Gender','family_history_with_overweight','FAVC','CAEC','SMOKE','CALC','MTRANS','Age','Height','Weight','FCVC','NCP','CH2O','FAF','TUE']
predecir=['NObeyesdad']
preprocesador1 = make_column_transformer(    
    (OneHotEncoder(),predecir),('drop',eliminar)
    )
DataSet1=preprocesador1.fit_transform(datos)
DataSet1=DataSet1.toarray()
nombres1 = preprocesador1.transformers_[0][1].get_feature_names(predecir)
DatasetPreprocesado1 = pd.DataFrame(data=DataSet1,columns=nombres1)
DatasetPreprocesado1.head()

,NObeyesdad_Insufficient_Weight,NObeyesdad_Normal_Weight,NObeyesdad_Obesity_Type_I,NObeyesdad_Obesity_Type_II,NObeyesdad_Obesity_Type_III,NObeyesdad_Overweight_Level_I,NObeyesdad_Overweight_Level_II
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,1.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,1.0


## DIVISION EN TRAIN Y TEST

In [ ]:
entradas=['Age', 'Height', 'Weight', 'FCVC', 'NCP', 'CH2O', 'FAF', 'TUE',
       'Gender_Female', 'Gender_Male', 'family_history_with_overweight_no',
       'family_history_with_overweight_yes', 'FAVC_no', 'FAVC_yes',
       'CAEC_Always', 'CAEC_Frequently', 'CAEC_Sometimes', 'CAEC_no',
       'SMOKE_no', 'SMOKE_yes', 'CALC_Always', 'CALC_Frequently',
       'CALC_Sometimes', 'CALC_no', 'MTRANS_Automobile', 'MTRANS_Bike',
       'MTRANS_Motorbike', 'MTRANS_Public_Transportation', 'MTRANS_Walking']
salidas=['NObeyesdad_Insufficient_Weight', 'NObeyesdad_Normal_Weight',
       'NObeyesdad_Obesity_Type_I', 'NObeyesdad_Obesity_Type_II',
       'NObeyesdad_Obesity_Type_III', 'NObeyesdad_Overweight_Level_I',
       'NObeyesdad_Overweight_Level_II']
y = DatasetPreprocesado1[salidas].values
X_train, X_test, y_train, y_test = train_test_split(DatasetPreprocesado[entradas], y, test_size=.2, random_state=42)

## ALMACENAMIENTO Y CARGA DE MODELOS DE REDES NEURONALES

In [ ]:
def guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos):
    print("Guardando Red Neuronal en Archivo")  
    # serializar modelo a JSON

    # Guardar los Pesos (weights)
    model.save_weights(nombreArchivoPesos+'.h5')

    # Guardar la Arquitectura del modelo
    with open(nombreArchivoModelo+'.json', 'w') as f:
        f.write(model.to_json())

    print("Red Neuronal Grabada en Archivo")   
    
def cargarRNN(nombreArchivoModelo,nombreArchivoPesos):
        
    # Cargar la Arquitectura desde el archivo JSON
    with open(nombreArchivoModelo+'.json', 'r') as f:
        model = model_from_json(f.read())

    # Cargar Pesos (weights) en el nuevo modelo
    model.load_weights(nombreArchivoPesos+'.h5')  

    print("Red Neuronal Cargada desde Archivo") 
    return model

## DISEÑO DE RED NEURONAL DE PRUEBA Y EVALUACION


In [ ]:
#Construcción del Modelo o Arquitectura de Redes Neoronales
model = Sequential()

#La primera capa Dense recibe el numero de variables, que es la segunda dimensión de la matriz X, esto es X_train.shape[1]
#La primera capa tiene 32 neuronas. La función de activación es la función rectificadora.
model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
#La segunda capa tiene 64 neuronas. La función de activación es la función rectificadora.
model.add(Dense(64, activation='relu'))
#La capa de salida tiene 1 neurona. La capa de salida debe tener la misma dimensión como de cantidad de salidas queremos,
#por ejemplo, en este caso la salida "Survived" solo requiere 0 y 1. Puesto que 0 o 1 ocupan solo un valor dentro de cada dato,
#entonces 1 neurona es suficiente. La función de activación es sigmoide para clasificación por probabilidad.
model.add(Dense(7, activation='softmax'))



#Como tenemos dos posibles salidas "0 o 1", vamos a escoger que el error lo trate como una clasificación binaria, 
#el optimizador será nuestra función derivada que nos ayudará a determinar hacia donde mover los pesos.
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy']) #ADADELTA: An Adaptive Learning Rate Method

#imprimir arquitectura de la red
model.summary()

#Entrenamiento: 

#Entrenaremos por 100 epochs, el batch_size es un argumento importante, porque representa cada cuántos datos va a actualizar
#los pesos. Este es el método del gradiente descendiente estocástico que hace el proceso más eficiente y preciso.
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)
score = model.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
score = model.evaluate(X_test, y_test, verbose=0)
print('Resultado en Test:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#mostrar pesos de la red
#print(model.get_weights())

#Guardar pesos y la arquitectura de la red en un archivo 

nombreArchivoModelo='arquitectura_prueba'
nombreArchivoPesos='pesos_prueba'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

#Cargar pesos y la arquitectura
model2=cargarRNN(nombreArchivoModelo,nombreArchivoPesos) 

model2.compile(loss='categorical_crossentropy', optimizer='adadelta', metrics=['acc']) #ADADELTA: An Adaptive Learning Rate Method
score = model2.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model2.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model2.metrics_names[1], score[1]*100))

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_3 (Dense)              (None, 32)                960       
_________________________________________________________________
dense_4 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_5 (Dense)              (None, 7)                 455       
Total params: 3,527
Trainable params: 3,527
Non-trainable params: 0
_________________________________________________________________
Resultado en Train:
accuracy: 93.66%
Resultado en Test:
accuracy: 92.20%
Guardando Red Neuronal en Archivo
Red Neuronal Grabada en Archivo
Red Neuronal Cargada desde Archivo
Resultado en Train:
acc: 93.66%
Resultado en Test:
acc: 92.20%


## PREDICCION - USO DEL MODELO DE PRUEBA 

In [ ]:
def predict(Gender='Male',Age=21,Height=1.62,Weight=64,family_history_with_overweight='yes',FAVC='no', FCVC=2.0,NCP=3.0,CAEC='Sometimes',SMOKE='no', CH2O=2.0,SCC='no',FAF=0.0,TUE=1.0,CALC='no',MTRANS='Public_Transportation'):
    cnames = ['Gender','Age','Height','Weight','family_history_with_overweight','FAVC','FCVC','NCP','CAEC','SMOKE','CH2O','SCC','FAF','TUE','CALC','MTRANS','NObeyesdad']
    data = [[Gender,Age,Height,Weight,family_history_with_overweight,FAVC,FCVC,NCP,CAEC,SMOKE,CH2O,SCC,FAF,TUE,CALC,MTRANS,'nada']]
    my_X = pd.DataFrame(data=data, columns=cnames)
    my_X = preprocesador.transform(my_X)    
    return model.predict_classes(my_X)

In [ ]:
predict(Gender='Female',Age=21,Height=1.62,Weight=64,family_history_with_overweight='yes',FAVC='no', FCVC=2.0,NCP=3.0,CAEC='Sometimes',SMOKE='no', CH2O=2.0,SCC='no',FAF=0.0,TUE=1.0,CALC='no',MTRANS='Public_Transportation')

Instructions for updating:
Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).


array([1])

## DISEÑO DE EXPERIMENTOS - OPTIMIZACION Y FINE TUNING 

In [ ]:
#Evaluación del modelo original: Esto puede tardar unos 30 segundos. El modelo anterior entrena 1 sóla vez, ahora haremos una
#media de entrenamientos con el modelo.
def build_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dense(64, activation='relu'))
    model.add(Dense(7, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

#El modelo se pasa como parámetro
estimator = KerasClassifier(build_fn=build_model, epochs=100, batch_size=64) 
#cv es la cantidad de veces de entrenamiento del modelo
#n_jobs es para ocupar mas de un procesador. El parámetro -1 indica que queremos utilizar todos los procesadores disponibles
accuracies=cross_val_score(estimator, X_train, y_train, cv=10, n_jobs=-1)
mean_acc=accuracies.mean()
print('Precision media: ', mean_acc)
##Como resultado al proceso de entrenamiento, tenemos un accuracy en promedio aprox. de: 0.79. Intentaremos mejorarlo.
#Cambien optimizer='adadelta' a optimizer='adam' o a 'rmsprop'

model=build_model() 
model.fit(X_train, y_train, epochs=100, batch_size=64, verbose=0)
model.summary()
print('Resultado en Train:')
score = model.evaluate(X_train, y_train, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

nombreArchivoModelo='arquitectura_base'
nombreArchivoPesos='pesos_base'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

Precision media:  0.9656558215618134
Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_6 (Dense)              (None, 32)                960       
_________________________________________________________________
dense_7 (Dense)              (None, 64)                2112      
_________________________________________________________________
dense_8 (Dense)              (None, 7)                 455       
Total params: 3,527
Trainable params: 3,527
Non-trainable params: 0
_________________________________________________________________
Resultado en Train:
acc: 99.88%
Resultado en Test:
acc: 95.98%
Guardando Red Neuronal en Archivo
Red Neuronal Grabada en Archivo


In [ ]:
#1. Compilación: Prueba de mejores parámetros batch_size, epochs y optimizer
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.

def build_model(optimizer):
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(7, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model


#parámetros que queremos probar, y sus valores 
#probaremos con batch_size, epochs, y optimizador, con el fin de encontrar la mejor combinación entre estos tres parámetros.
parameters = parameters = {'batch_size': [16,32],
             'epochs':[100,500],
             'optimizer': ['adadelta', 'rmsprop']}

estimator = KerasClassifier(build_fn=build_model, verbose=0)
#Ahora no le pasamos los parámetros al KerasClasifier, porque se los pasaremos a través de GridSearchCV
#el argumento verbose=0 es para que no muestre salida, si lo dejamos en cero, no mostrará la barra de progreso del entrenamiento
#GridSearchCV: recibe como parámetros nuestro modelo, nuestros parámetros, la medida sobre la que queremos comparar, y la 
#cantidad de veces que lo entrenará para sacar la media de accuracy.
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
#grid_search.best_params_
print(grid_search.best_params_)
#Un ejemplo de resultados es: {'batch_size': 16, 'epochs': 100, 'optimizer': 'rmsprop'}
#Esto indica que el optimizador "adadelta" no es adecuado. Y es que este optimizador NO sirve para este tipo de problemas.

ValueError: ignored

In [ ]:
#2. Densidad de las capas de neuronas
#Notemos que se incluyen los mejores parámetros del paso de optimización anterior (batch_size, epochs y optimizer)
#Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(l1, l2):
    model = Sequential()
    model.add(Dense(l1, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(l2, activation='relu'))
    model.add(Dropout(0.1))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'l1':[16,32,64,128,256],
                           'l2':[16,32,64,128,256]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=10,n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)

#Resultados: {'l1': 32, 'l2': 16}
#Los resultados indican que hubo un error en la red original, las capas van desde la más densa, a la menos densa.

ValueError: ignored

In [ ]:
#3. Proceso con dropouts: apagar un porcentaje de neuronas al azar con el fin de que las neuronas no se vuelvan tan 
#dependientes de los datos.
##Esto recomiendo probarlo con Google Colab, puesto que se necesita 16GB en RAM y puede llegar a tardar unos 30min.
def build_model(d1, d2):
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(d1))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(d2))
    model.add(Dense(7, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

parameters = parameters = {'d1':[0.1,0.2,0.3],
                            'd2':[0.1,0.2,0.3]}

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
grid_search = GridSearchCV(estimator=estimator, param_grid=parameters, scoring='accuracy', cv=3,n_jobs=-1)
grid_search.fit(X_train, y_train)
grid_search.best_params_
print(grid_search.best_params_)

#Resultados: {'d1':0.2, 'd2':0.3}

ValueError: ignored

In [ ]:
#Evaluación del Modelo Final

finalModel = Sequential()
#Finalmente, veamos como mejoró nuestro modelo, vamos a repetir el proceso de la validación cruzada.
#Esto puede probarse localmente. Ya con los mejores parámetros evaluamos la red neuronal. Puede tardar un minuto.
def build_model():
    model = Sequential()
    model.add(Dense(32, input_shape=(X_train.shape[1],), activation='relu'))
    model.add(Dropout(0.2))
    model.add(Dense(16, activation='relu'))
    model.add(Dropout(0.3))
    model.add(Dense(7, activation='sigmoid'))
    model.compile(loss='categorical_crossentropy', optimizer='rmsprop', metrics=['acc'])
    return model

estimator = KerasClassifier(build_fn=build_model, verbose=0, batch_size=16, epochs=100)
accuracies = cross_val_score(estimator, X_train, y_train, cv=10, n_jobs=-1)
mean_acc = accuracies.mean()
std_acc = accuracies.std()
print('accuracies: ')
print(accuracies)
print('Precisión media: ', mean_acc)
print('Desviación media: ',std_acc)
#Y el resultado es:
#Precision media aprox.: 0.8067
#Pasamos de un 50% precisión a un 80% de precisión, por lo que se recomienda: 
#hacer siempre el proceso de fine tunning, porque ayudará a crear modelos correctos en la mayoría de los casos.

model=build_model() 
model.fit(X_train, y_train, epochs=100, batch_size=16, verbose=1)
model.summary()

score = model.evaluate(X_train, y_train, verbose=0)
print('Resultado en Train:')
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

#Fase de Testing
print('Resultado en Test:')
score = model.evaluate(X_test, y_test, verbose=0)
print("%s: %.2f%%" % (model.metrics_names[1], score[1]*100))

nombreArchivoModelo='arquitectura_optimizada'
nombreArchivoPesos='pesos_optimizados'
guardarRNN(model,nombreArchivoModelo,nombreArchivoPesos)

accuracies: 
[0.95266271 0.98224854 0.94082838 0.95266271 0.87573963 0.91715974
 0.96449703 0.9585799  0.98214287 0.98214287]
Precisión media:  0.950866436958313
Desviación media:  0.03173948818430531
Epoch 1/100
106/106 [==============================] - 0s 3ms/step - loss: 1.9105 - acc: 0.1759
Epoch 2/100
106/106 [==============================] - 0s 3ms/step - loss: 1.7397 - acc: 0.2441
Epoch 3/100
106/106 [==============================] - 0s 3ms/step - loss: 1.5525 - acc: 0.3146
Epoch 4/100
106/106 [==============================] - 0s 2ms/step - loss: 1.4178 - acc: 0.3945
Epoch 5/100
106/106 [==============================] - 0s 2ms/step - loss: 1.3028 - acc: 0.4271
Epoch 6/100
106/106 [==============================] - 0s 3ms/step - loss: 1.2172 - acc: 0.4650
Epoch 7/100
106/106 [==============================] - 0s 3ms/step - loss: 1.1223 - acc: 0.5018
Epoch 8/100
106/106 [==============================] - 0s 2ms/step - loss: 1.0378 - acc: 0.5249
Epoch 9/100
106/106 [==========

#Conclusiones
Debido a la naturaleza de categorizacion multiple que se presenta en el data set, varias medidas de mejora dentro de la red neuronal no se euncuentral disponibles y por lo tanto se procede a evaluar solo con otra media.
Al ver una medida de mas del 90% de accuracy n el modelo inicial y el mejorado se puede argumentar que esto bastaria para el proposito buscado.
Ademas esto representa una evolucion deseada sin llegar a un sobre entrenamiento.